# Practical session 3: Parallel behaviors and more sensing abilities

In the last practical session, we saw how to define, attach, start, stop and detach a behavior on an E-Puck robot. We implemented three distinct behaviors: `slow_down`, `fear` and `aggression`.

In this section we will see more sensing abilities the E-Puck is equipped with, will define new behaviors using them and will see how to deal with multiple behaviors running in parallel on the same robot. At the end of the session, we will also see how to attach those behaviors on multiple robots interacting together within a V-REP scene.

Open V-REP and load the scene `epuck-scene-3.ttt` located in the directory `Documents/robotics/pyvrep_epuck/vrep-scenes`.

As usual, open the simulator session:

In [76]:
from vivarium.controllers.notebook_controller import NotebookController
controller = NotebookController()

In [77]:
controller.run()

## Selectively detecting scene objects

To define a repertoire of interesting behaviors, we need the robot to selectively sense the proximity of different types of objects. For example, we might want to define a behavior for obstacle avoidance and another one for attraction towards mates. The first behavior will require the proximity from walls and pillars, whereas the second will require the proximity from other robots (although there is only one robot in the scene for now, we'll add more at the end of this session). 

We can filter the result returned by the E-Puck's proximeters by providing the argument `tracked_objects` to the `prox_activations` function:

In [78]:
agent = controller.agents[0]

In [79]:
agent.infos(full_infos=False)

Agent Overview:
--------------------
Entity Type: AGENT
Position: x=68.91, y=71.65
Behaviors: None
Sensors: Left=0.71, Right=0.71
Motors: Left=0.00, Right=0.00


In [80]:
# for this notebook

agents_id = 0 # blue 
ressources_id = 1 # green
obstacles_id = 2 # red

In [81]:
left, right = agent.sensors()
print(left, right)

0.7127175331115723 0.713765561580658


In [82]:
left, right = agent.sensors(sensed_entities=[obstacles_id])
print(left, right)

0.7127175331115723 0


In [83]:
left, right = agent.sensors(sensed_entities=[agents_id])
print(left, right)

0 0


Executing the cell above will return the proximeter activations only for the `Cup` objects (the kind of trash bins in the V-REP scene). Give it a try by moving a cup in the detection area of the proximiters and re-executing the cell above to observe the change in the returned values. You can also check that the proximeter activations are not modified by other objects such as pillars.

The `tracked_objects` argument requires a list of strings (`["Cup"]` in the example above). In Python, a list is a collection of values separated by commas and surrounded by square bracket: `["Cup"]` is therefore a list of only one element (the string `"Cup"`), whereas `["Cup", "ePuck"]` is a list of two elements (the strings `"Cup"` and `"ePuck"`). 
The `tracked_objects` argument, as its name indicates, sets the objects to be tracked by the proximeters. Each object in a V-REP scene has a name, which is shown when you select an object by clicking on it in the interface. You can also inspect the names of all the objects in the `Scene hierarchy` panel on the left (if not visible, you can activate in the `Tools` menu). For example, we see that the cups have the names `Cup`, `Cup0`, `Cup1` etc ..

In the cell above, `tracked_objects=["Cup"]` means *only return the proximeter activations of objects having their names starting with `"Cup"`*. Since only cups have their names starting with `Cup`, it will return the proximeter activation only for cups, not considering e.g. walls and pillars.

**Q1:** Write the code printing the proximeter activations for walls and pillars and test that it works as expected by placing your epuck close to those objects and verifying the returned values:

In order to track several types of objects at the same time, we can pass several strings to `tracked_objects`. For example, if we want to track both cups and trees, we will write:

In [84]:
left, right = agent.sensors(sensed_entities=[ressources_id, obstacles_id])
print(left, right)

0.7127175331115723 0.713765561580658


This is because all trees in the scene have their name starting with `Tree` (`Tree`, `Tree#0`, `Tree#1` etc..., as shown in the `Scene hierarchy` panel).

**Q2:** Define an `obstacle_avoidance` behavior. Obstacles are walls, pillars and trees, but not cups.  The robot has to turn in the direction opposite to the obstacle, with its speed inversely proportional to the proximeter activations (the closer an obstacle, the lower the speed). 
*Tip:* it is similar to the `shyness` behavior of [Braitenberg vehicles](https://docs.google.com/presentation/d/1s6ibk_ACiJb9CERJ_8L_b4KFu9d04ZG_htUbb_YSYT4/edit#slide=id.g31e1b425a3_0_0).

In [85]:
def obstacle_avoidance(epuck):
    # Write your code below
    pass


In [86]:
def obstacle_avoidance(agent):
    left, right = agent.sensors(sensed_entities=[obstacles_id])
    return 1. - right, 1. - left


Remember that to test a behavior, you first have to detach all behaviors that could still be attached to the E-Puck, then to attach and start the new one, that is:

In [87]:
agent.detach_all_behaviors()
agent.attach_behavior(obstacle_avoidance)
agent.start_all_behaviors()

In [88]:
agent.detach_all_behaviors()

In [91]:
agent.check_behaviors()

No behaviors attached


In [90]:
agent.stop_motors()

## Environmental dynamics

Until now the environment in which the E-Puck is evolving is quite static: although some objects can be pushed by the robot (e.g. the cups), there is nothing that appears or disappears in the environment. We are now going to see how we can generate food sources appearing at random positions in the environment and disappearing whenever a robot eat them. A food source is modeled as a V-REP `Sphere` object, meaning that it can roll on the floor. Making such spheres to appear at regular time intervals and at random positions in the environment is done with:

In [15]:
# TODO : might need to use a thread
def start_sphere_apparition(period=5):
    print(controller.all_entities)
    return 

start_sphere_apparition(1)

[<vivarium.controllers.notebook_controller.Agent object at 0x7a0261f36f50>, <vivarium.controllers.notebook_controller.Agent object at 0x7a0261f4b460>, <vivarium.controllers.notebook_controller.Object object at 0x7a0261f4b070>, <vivarium.controllers.notebook_controller.Object object at 0x7a0261f6f8b0>, <vivarium.controllers.notebook_controller.Object object at 0x7a0261f6f8e0>, <vivarium.controllers.notebook_controller.Object object at 0x7a0261f6d1e0>, <vivarium.controllers.notebook_controller.Object object at 0x7a0261f6f940>, <vivarium.controllers.notebook_controller.Object object at 0x7a0261f6f9a0>, <vivarium.controllers.notebook_controller.Object object at 0x7a0261f6fa00>, <vivarium.controllers.notebook_controller.Object object at 0x7a0261f6fa60>, <vivarium.controllers.notebook_controller.Object object at 0x7a0261f6fac0>, <vivarium.controllers.notebook_controller.Object object at 0x7a0261f6fb20>, <vivarium.controllers.notebook_controller.Object object at 0x7a0261f6fb80>, <vivarium.con

In [92]:
controller.objects

existing_objects = {}
non_existing_objects = {}

for i, obj in enumerate(controller.objects):
    print(obj.exists)
    if obj.exists:
        existing_objects[i] = obj
    else:
        non_existing_objects[i] = obj

print(existing_objects.keys())
print(non_existing_objects.keys())

True
True
True
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
dict_keys([0, 1, 2, 15, 16, 17, 18, 19])
dict_keys([3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14])


In [93]:
import time

In [94]:
def spawn_object(object_id, position=None):
    if object_id in existing_objects:
        print("Object already exists")
        return
    obj = controller.objects[object_id]
    if position is not None:
        # TODO : Add a check to prevent from having a similar position among other entities
        obj.x_position = position[0]
        obj.y_position = position[1]
    obj.exists = True
    existing_objects[object_id] = obj
    del non_existing_objects[object_id]

def remove_object(object_id):
    if object_id in non_existing_objects:
        print("Object already removed")
        return
    obj = controller.objects[object_id]
    obj.exists = False
    non_existing_objects[object_id] = obj
    del existing_objects[object_id]

In [95]:
import numpy as np

In [107]:
stop_signal = False

In [104]:
import threading

In [127]:
def object_apparition(period=2):
    global stop_signal
    while not stop_signal and non_existing_objects:
        ids = list(non_existing_objects.keys())
        id = np.random.choice(ids)
        spawn_object(id)
        time.sleep(period)

def start_object_apparition(period=2):
    global stop_signal
    stop_signal = False
    thread = threading.Thread(target=object_apparition, args=(period,))
    thread.start()

def stop_object_apparition():
    global stop_signal
    stop_signal = True



In [139]:
agent.left_motor = 0.

In [138]:
stop_object_apparition()

In [137]:
start_object_apparition(1)

In [129]:
for i in range(len(controller.objects)):
    remove_object(i)

Object already removed
Object already removed
Object already removed
Object already removed
Object already removed
Object already removed


In [44]:
obj = controller.objects[0]
print(obj.__dict__)
print(obj.x_position)

{'config': ObjectConfig(color='#008000', diameter=5.0, exists=False, friction=0.10000000149011612, idx=2, mass_center=1.0, mass_orientation=0.125, name='ObjectConfig00006', orientation=5.935915470123291, x_position=175.28672790527344, y_position=39.2646369934082), '_routines': {}, 'user_events': {}, 'etype': <EntityType.OBJECT: 1>}
175.28672790527344


In [68]:
spawn_object(15, position=[80, 70])

obj.config.x_position = 80, obj.config.y_position = 70


In [69]:
print(controller.objects[15].x_position)
print(controller.objects[15].y_position)

80
70


In [66]:
remove_object(15)

In [28]:
agent.left_motor = 0.0

In [ ]:
controller.start_sphere_apparition(period=5.)

where `period` indicates the time interval at which spheres will appear (here every 5 seconds). In order to stop sphere apparition:

In [ ]:
simulator.stop_sphere_apparition()

We can also specify in which area spheres appear through the `min_pos` and `max_pos` arguments:

In [ ]:
simulator.start_sphere_apparition(period=5, min_pos=[-2, -1, 1], max_pos=[2., 0.5, 2])

The cell above will generate spheres at random 3D positions $(x, y, z)$ in the scene with $x\in[-2, 2]$, $y\in[-1, 0.5]$ and $z\in[1, 2]$ (analyze the cell above to understand how the `min_pos` and `max_pos` arguments are converted in $(x, y, z)$ intervals and ask us if it is not clear). You can check how the $x, y, z$ axes are oriented at the bottom-right corner of the V-REP scene (try to rotate the scene while looking at it). When selecting an object, you can check the coordinate of its center in the text located in the top-left corner of the scene (`Last selected object position`). The center of the scene is at $x=0, y=0, z=0$. The floor is contained in (approximately) $x\in[-2.5, 2.5]$ and $y\in[-2.5, 2.5]$, with $z=0$ on the surface. 

**Q3:** Write the code that makes sphere appearing just above one of the trees and so that they then roll in various directions:

Whenever the E-Puck is touching a sphere it will "eat" it, meaning that the sphere will disappear from the environment. You can see it by manually moving the E-Puck close to a sphere (the sphere should disappear, although this might take some time). Note however that the spheres will disappear only if the sphere apparition is activated (i.e. if you haven't executed `simulator.stop_sphere_apparition()` as the last command: in that case, you will have to re-execute `simulator.start_sphere_apparition(period=5.)` as above).

If no E-Puck is eating the spheres, you might end up with a large number of spheres occupying the environment and this could dramatically impair the V-REP performances. In that case, clean the environment by closing and restarting the session as explained at the beginning of the notebook. A good practice to avoid the proliferation of spheres is to increase the period (e.g.`simulator.start_sphere_apparition(period=20.)`) and to stop sphere apparition whenever you don't need it.

## Combining behaviors

**Q4:** Define a behavior allowing the robot to catch food sources, let's call it `foraging`. The robot has to orient itself toward food sources, with a speed proportional to the proximiter activations (the closer the food source, the higher the speed) *Tip 1:* It's similar to the `aggression` behavior. *Tip 2:* Generated spheres have their names starting by "Sphere" (you can see it in V-REP in the scene hierarchy panel).

In [ ]:
def foraging(epuck):
    # Write your code below


In [140]:
def foraging(agent):
    left, right = agent.sensors(sensed_entities=[ressources_id])
    return right, left


Attach and start the behavior on the E-Puck: 

In [146]:
# First detach previous behaviors that might still be attached to the robot
agent.detach_all_behaviors()

# Write the code to attach and start the `foraging` behavior below
agent.attach_behavior(foraging)
agent.start_all_behaviors()

In [147]:
agent.check_behaviors()

Available behaviors: ['foraging']
active behaviors: ['foraging']


Whenever a sphere is detected by the proximeters, the robot should go towards it. However, if at one point the proximeters don't detect any sphere, the robot will probably stop (depending on how you have defined the behavior). The only event that could make the robot move again would be a sphere that rolls within the proximeter detection area, which is not very likely to happen (and therefore quite a bad option if the survival of the robot depends on its foraging abilities). A solution to avoid such a blocking situation is to combine the `foraging` behavior with another one that keeps the robot in movement, as it is for example the case of the `obstacle_avoidance` behavior we have defined before. Let's attach and start the `obstacle_avoidance` behavior, but this time without detaching the previously attached `foraging` behavior:

In [148]:
agent.attach_behavior(obstacle_avoidance)
agent.start_behavior(obstacle_avoidance)

Since we haven't detached the previous behavior, the robot is now executing two behaviors in parallel. This can be checked with:

In [149]:
agent.check_behaviors()

Available behaviors: ['foraging', 'obstacle_avoidance']
active behaviors: ['foraging', 'obstacle_avoidance']


which tells us that both behaviors are attached and started. In V-REP, you can see that the robot is now both foraging and avoiding obstacles. For doing so, the motor activation sent to each wheel corresponds to the average of the motor activation returned by each behavior (this averaging is implemented internally, you don't need to worry about it).

## Floor sensor

First restart the session to clean the environment. Close it:

In [9]:
from simulator_interface import open_session, close_session
close_session(simulator)

Wait for the closing of the session (i.e. until the star on the left of the above cell disappears). 

Then reopen the session:

In [10]:
simulator, epuck = open_session()

The E-Puck is equipped with one more sensor that we haven't seen yet and that could be useful for some projects: a floor sensor. It is able to detect a line drawn on the floor if it is placed on it. Read this sensor by executing:

In [ ]:
floor_left, floor_middle, floor_right = epuck.floor_sensor()
print(floor_left, floor_middle, floor_right)

This returns three values, corresponding to the activity of the three floor sensors placed at the bottom of the robot: the first one on the left side, the second one on the middle side and the third one on the right side. Each of these sensors return 0.0 if no line is detected and 1.0 if a line is detected. Position the E-Puck on top of the black curved line which is on the scene floor such that when you execute the cell above, one of the sensors returns 1.0.(you will have to be quite precise, you can zoom in to the robot to have a better view).

**Q5:** Define a `line_following` behavior, such that when the robot is positioned on the line which is drawn on the floor, it moves forward and keeps following it. *Tip 1*: reduce the maximum speed of the robot to obtain a more precise behavior (this is already done at the top of the code cell below). *Tip 2*: ask yourself the following questions: what should be the left and right wheel activations when only the left floor sensor is activated? when only the right floor sensor is activated? when only the middle floor sensor is activated? Write the behavior code in the cell below:

In [17]:
# First reduce the maximum speed
epuck.max_speed = 1.0

def line_following(epuck):
    # Write your code below
    

Then, as usual, detach all previously attached behaviors, attach the new one and start it.

## Dealing with multiple robots

This section explains how to deal with multiple robots and how to attach different behaviors to them.

First close the current session:

In [6]:
close_session(simulator)

Then restart the notebook (`Kernel -> Restart`).

Select the E-Puck robot by clicking on it in the scene. Copy-paste it, either using the `Edit` menu, or by pressing the usual editing shortcut `Ctrl-C` then `Ctrl-V`. A new E-puck will be placed in the scene at the exact same position as the previous one. Drag and drop this new robot to another position. Now you should see two robots in the scene.

Re-open a session, this time requesting the references to two E-Pucks instead of one, by executing:

In [1]:
from simulator_interface import open_session, close_session

simulator, epuck1, epuck2 = open_session(n_epucks=2)

In [150]:
for ag in controller.agents:
    ag.exists = True

In [151]:
agent_1 = controller.agents[0]
agent_2 = controller.agents[1]

Now you have access to the two E-Pucks through the variables `epuck1` and `epuck2` (these variables names are arbitrary, you can choose whatever you want, e.g. `predator` and `prey`). You can attach and start behaviors on each E-Puck independently, in the same way as you did before, simply using either the `epuck1` and `epuck2` variables instead of only `epuck` one as before.

As an example, let's say we want to attach the `obstacle_avoidance` behavior we have defined above to `epuck1`, and both the `obstacle_avoidance` and the `foraging` behaviors to `epuck2`. 
Since we have restarted the notebook, you first need to re-execute the cells defining both behaviors above (i.e. the cells starting with `def obstacle_avoidance(epuck):` and `def foraging(epuck):`).

In [152]:
agent_2.attach_behavior(obstacle_avoidance)
agent_2.attach_behavior(foraging)
agent_2.start_all_behaviors()

The code will be:

In [ ]:
# As usual, we detach the possibly already running behaviors.
# Since we now have two epucks, called `epuck1` and `epuck2`, we have to do it on each of them.
agent_1.detach_all_behaviors()

# Then we attach the obstacle avoidance behavior to epuck1:
agent_1.attach_behavior(obstacle_avoidance, freq=10)

# Then we attach both the obstacle avoidance and the foraging behavior to epuck2:
agent_2.attach_behavior(obstacle_avoidance, freq=10)
agent_2.attach_behavior(foraging, freq=10)

# Finally, we start the attached behaviors on each epuck
agent_1.start_all_behaviors()  # This will start obstacle_avoidance on epuck1 (because it is the only behavior we have attached to epuck1)
agent_2.start_all_behaviors()  # This will start both obstacle_avoidance and foraging on epuck2 (because we have attached both behaviors on epuck2)

**Q5:** Implement the `fear` and `aggression` behaviors so that they are directed only toward the other E-Puck, using the `tracked_objects` argument of the `prox_activations` function as we have seen above. Then attach both the `obstacle_avoidance` and the `aggression` behaviors to one E-Puck, and both the `obstacle_avoidance` and the `fear` behaviors on the second. If you did it well, you should observe a simple "prey-predator" interaction, where `epuck1` tries to catch `epuck2` and `epuck2` tries to escape from `epuck1`.

In [153]:
for ag in controller.agents:
    ag.detach_all_behaviors()
    ag.stop_motors()

In [154]:
def aggression(agent):
    left, right = agent.sensors(sensed_entities=[agents_id])
    return right, left

def fear(agent):
    left, right = agent.sensors(sensed_entities=[agents_id])
    return left, right

In [155]:
agent_1.attach_behavior(aggression)
agent_1.attach_behavior(obstacle_avoidance)
agent_1.start_all_behaviors()

agent_2.attach_behavior(fear)
agent_2.attach_behavior(obstacle_avoidance)
agent_2.start_all_behaviors()

In [ ]:
# Your code here
